In [156]:
from scipy.stats import multivariate_normal
from random import randint
import numpy as np

In [51]:
x, y = [], []
file = open("2gaussian.txt", 'r')
k = 2
for line in file:
    x.append(float(line.split()[0]))
    y.append(float(line.split()[1]))

In [179]:
u = [[2,2], [6,3]]
cov_mat = [[[1,0],[0,2]], [[1,0.9], [0.9,1]]]
weights = {0: 0.48733333333333334, 1: 0.5126666666666667}
pi_0 = {}
pi_1 = {}

In [158]:
def probablity(data_point, u, cov_mat):
    var = multivariate_normal(mean=u, cov=cov_mat)
    prob = var.pdf([data_point[0], data_point[1]])
    return prob

In [159]:
def expectation():
    for i in range(0, len(x)):
        data_point = (x[i], y[i])
        probs = []
        for j in range(0, k):
            prob = probablity(data_point, u[j], cov_mat[j])
            if j == 0:
                pi_0[data_point] = prob * weights[0]
            else:
                pi_1[data_point] = prob * weights[1]
            probs.append((prob, j))
        denom = 0
        for prob in probs:
            label = prob[1]
            denom += prob[0] * weights[label]

        pi_0[data_point] /= denom
        pi_1[data_point] /= denom
        

In [161]:
def maximization():
    weights[0] = sum(list(pi_0.values())) / len(x)
    weights[1] = sum(list(pi_1.values())) / len(x)
    sum_u0_x = 0
    sum_u0_y = 0
    sum_u1_x = 0
    sum_u1_y = 0
    for i in range(0, len(x)):
        data_point = (x[i], y[i])
        p0 = pi_0[data_point]
        p1 = pi_1[data_point]
        sum_u0_x += (p0 * x[i])
        sum_u0_y += (p0 * y[i])
        sum_u1_x += (p1 * x[i])
        sum_u1_y += (p1 * y[i])
    sum_u0_x /= sum(list(pi_0.values()))
    sum_u0_y /= sum(list(pi_0.values()))
    sum_u1_x /= sum(list(pi_1.values()))
    sum_u1_y /= sum(list(pi_1.values()))
    u[0][0] = sum_u0_x
    u[0][1] = sum_u0_y
    u[1][0] = sum_u1_x
    u[1][1] = sum_u1_y
    update_cov_mat()

In [162]:
def update_cov_mat():
    final_cov_mat0 = [[0,0], [0,0]]
    final_cov_mat1 = [[0,0], [0,0]]
    for i in range(len(x)):
        data_point = (x[i], y[i])
        prob0 = pi_0[data_point]
        prob1 = pi_1[data_point]
        
        factor01 = [data_point[0] - u[0][0], data_point[1] - u[0][1]]
        factor01 = np.array(factor01).reshape(1,-1)
        factor02 = np.dot(factor01.reshape(-1,1), factor01)
        factor03 = prob0 * factor02
        
        for m in range(k):
            for n in range(k):
                final_cov_mat0[m][n] += factor03[m][n]
        
        factor11 = [data_point[0] - u[1][0], data_point[1] - u[1][1]]
        factor11 = np.array(factor11).reshape(1,-1)
        factor12 = np.dot(factor11.reshape(-1,1), factor11)
        factor13 = prob1 * factor12
        
        for m in range(k):
            for n in range(k):
                final_cov_mat1[m][n] += factor13[m][n]
                
    for m in range(k):
        for n in range(k):
            final_cov_mat0[m][n] = final_cov_mat0[m][n] / sum(list(pi_0.values()))
    
    for m in range(k):
        for n in range(k):
            final_cov_mat1[m][n] = final_cov_mat1[m][n] / sum(list(pi_1.values()))
    
    cov_mat[0] = final_cov_mat0
    cov_mat[1] = final_cov_mat1

In [202]:
def repeat_untill_convergence():
    iterate = True
    i = 0
    while(iterate):
        mean1_x = u[0][0]
        mean1_y = u[0][1]
        mean2_x = u[1][0]
        mean2_y = u[1][1]
        print("Iteration: " + str(i))
        expectation()
        print("Expectation done")
        maximization()
        print("Maximization done")
        print("Mean 1: ", end = " ")
        print(u[0])
        print("Mean 2: ", end = " ")
        print(u[1])
        print("Covariance 1: ", end = " ")
        print(cov_mat[0])
        print("Covariance 2: ", end = " ")
        print(cov_mat[1])
        print("Weights: ", end = " ")
        print(weights)
        print("=============================================")
        i += 1
        if abs(u[0][0] - mean1_x) < 0.000000000001 and abs(u[0][1] - mean1_y) < 0.000000000001 and abs(u[1][0] - mean2_x) < 0.000000000001 and abs(u[1][1] - mean2_y) < 0.000000000001:
            iterate = False

In [176]:
repeat_untill_convergence()

Iteration: 0
Expectation done
Maximization done
Mean 1:  [2.9941286919428425, 3.0520969723220324]
Mean 2:  [7.013146686211203, 3.983133255383811]
Covariance 1:  [[1.0102290791745785, 0.02719195419449861], [0.02719195419449861, 2.9378254008738995]]
Covariance 2:  [[0.9747617641819106, 0.4974719184430906], [0.4974719184430906, 1.0011439556232402]]
Weights:  {0: 0.3347952368080084, 1: 0.6652047631919904}
Iteration: 1
Expectation done
Maximization done
Mean 1:  [2.994129706920353, 3.052096851591695]
Mean 2:  [7.0131472133395, 3.9831335565991792]
Covariance 1:  [[1.0102307563356907, 0.027191771284675835], [0.027191771284675835, 2.9378246126785967]]
Covariance 2:  [[0.9747608454283534, 0.49747139697282594], [0.49747139697282594, 1.0011435151312067]]
Weights:  {0: 0.33479540860569124, 1: 0.665204591394308}
Iteration: 2
Expectation done
Maximization done
Mean 1:  [2.9941303923557263, 3.052096770061032]
Mean 2:  [7.013147569319061, 3.9831337600155354]
Covariance 1:  [[1.0102318889596837, 0.0271

Expectation done
Maximization done
Mean 1:  [2.994131816235115, 3.052096600697207]
Mean 2:  [7.013148308805562, 3.9831341825780386]
Covariance 1:  [[1.0102342418058885, 0.027191391173401747], [0.027191391173401747, 2.9378229746635163]]
Covariance 2:  [[0.9747589361033109, 0.4974703132700405], [0.4974703132700405, 1.001142599712625]]
Weights:  {0: 0.3347957656328349, 1: 0.6652042343671651}
Iteration: 20
Expectation done
Maximization done
Mean 1:  [2.9941318168201083, 3.0520966006276167]
Mean 2:  [7.01314830910942, 3.98313418275166]
Covariance 1:  [[1.01023424277253, 0.027191391067984075], [0.027191391067984075, 2.9378229742092343]]
Covariance 2:  [[0.9747589355738, 0.49747031296949734], [0.49747031296949734, 1.0011425994587484]]
Weights:  {0: 0.3347957657318513, 1: 0.6652042342681466}
Iteration: 21
Expectation done
Maximization done
Mean 1:  [2.9941318172151736, 3.052096600580627]
Mean 2:  [7.013148309314597, 3.9831341828688895]
Covariance 1:  [[1.010234243425341, 0.027191390996793255],